## Cookie Factory Optimization Problem
Alex Szpakiewicz, Sara Thibierge, Léonard Roussard

In [2]:
# Library imports
import pandas as pd

### Data Loading

In [29]:
# Load the defects data from the provided CSV file
defects_file_path = 'defects.csv'
defects_data = pd.read_csv(defects_file_path)

In [24]:
biscuits = {
    'Biscuit_0': {'length': 4, 'value': 6, 'defect_thresholds': {'a': 4, 'b': 2, 'c': 3}},
    'Biscuit_1': {'length': 8, 'value': 12, 'defect_thresholds': {'a': 5, 'b': 4, 'c': 4}},
    'Biscuit_2': {'length': 2, 'value': 1, 'defect_thresholds': {'a': 1, 'b': 2, 'c': 1}},
    'Biscuit_3': {'length': 5, 'value': 8, 'defect_thresholds': {'a': 2, 'b': 3, 'c': 2}}
}

dough_length = 500

## Genetic Algorithm

In [25]:
import random
from deap import creator, base, tools, algorithms

# Define the individual and fitness function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize necessary components for the genetic algorithm
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, len(biscuits) - 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, dough_length)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Check if a biscuit can be placed at a given position considering defect thresholds
def can_place_biscuit(biscuit, position, defects):
    if position + biscuit['length'] > dough_length:
        return False 

    # Filter defects that fall within the biscuit's range
    relevant_defects = defects[(defects['x'] >= position) & (defects['x'] < position + biscuit['length'])]
    
    # Check if the biscuit's defect thresholds are exceeded
    for defect_class in biscuit['defect_thresholds']:
        max_allowed = biscuit['defect_thresholds'][defect_class]
        defects_count = relevant_defects[relevant_defects['class'] == defect_class].shape[0]
        if defects_count > max_allowed:
            return False
    
    return True

# Fitness function to evaluate each individual
def evaluate(individual):
    total_value = 0
    position = 0
    while position < dough_length:
        biscuit_idx = individual[position]
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            total_value += biscuit['value']
            position += biscuit['length']
        else:
            position += 1
    return total_value,

# Genetic operators
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(biscuits) - 1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Create initial population
population = toolbox.population(n=100)

# Run the genetic algorithm
ngen = 50  # Number of generations
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=False)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_fitness = evaluate(best_individual)[0]

# Get best individual's biscuit placement
biscuit_placement = []
position = 0
while position < dough_length:
    biscuit_idx = best_individual[position]
    biscuit_name = f'Biscuit_{biscuit_idx}'
    biscuit = biscuits[biscuit_name]
    if can_place_biscuit(biscuit, position, defects_data):
        biscuit_placement.append((biscuit_name, position, biscuit['value']))
        position += biscuit['length']
    else:
        position += 1
        
print(f"Total value of biscuits placed: {best_fitness}")
print(f"Biscuit placement: {biscuit_placement}")

/opt/homebrew/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/opt/homebrew/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Total value of biscuits placed: 715
Biscuit placement: [('Biscuit_1', 1, 12), ('Biscuit_1', 9, 12), ('Biscuit_3', 17, 8), ('Biscuit_0', 23, 6), ('Biscuit_1', 27, 12), ('Biscuit_0', 36, 6), ('Biscuit_1', 40, 12), ('Biscuit_2', 48, 1), ('Biscuit_0', 50, 6), ('Biscuit_0', 54, 6), ('Biscuit_0', 58, 6), ('Biscuit_3', 62, 8), ('Biscuit_1', 67, 12), ('Biscuit_0', 75, 6), ('Biscuit_3', 79, 8), ('Biscuit_3', 84, 8), ('Biscuit_1', 89, 12), ('Biscuit_1', 98, 12), ('Biscuit_3', 108, 8), ('Biscuit_0', 114, 6), ('Biscuit_0', 119, 6), ('Biscuit_1', 124, 12), ('Biscuit_1', 132, 12), ('Biscuit_0', 140, 6), ('Biscuit_0', 144, 6), ('Biscuit_1', 148, 12), ('Biscuit_0', 158, 6), ('Biscuit_0', 162, 6), ('Biscuit_3', 167, 8), ('Biscuit_0', 172, 6), ('Biscuit_1', 176, 12), ('Biscuit_0', 186, 6), ('Biscuit_2', 191, 1), ('Biscuit_2', 193, 1), ('Biscuit_1', 195, 12), ('Biscuit_0', 203, 6), ('Biscuit_0', 207, 6), ('Biscuit_3', 211, 8), ('Biscuit_0', 216, 6), ('Biscuit_0', 220, 6), ('Biscuit_0', 224, 6), ('Biscuit

In [19]:
len(biscuit_placement)

90

In [27]:
import pandas as pd

# Dough length
dough_length = 500


# Function to check if a biscuit can be placed at a given position considering defect thresholds
def can_place_biscuit(biscuit, position, defects):
    if position + biscuit['length'] > dough_length:
        return False  # Biscuit exceeds dough length

    # Filter defects that fall within the biscuit's range
    relevant_defects = defects[(defects['x'] >= position) & (defects['x'] < position + biscuit['length'])]

    # Check if the biscuit's defect thresholds are exceeded
    for defect_class in biscuit['defect_thresholds']:
        max_allowed = biscuit['defect_thresholds'][defect_class]
        defects_count = relevant_defects[relevant_defects['class'] == defect_class].shape[0]
        if defects_count > max_allowed:
            return False

    return True


import numpy as np



def optimize_biscuit_placement(biscuits, dough_length, defects):
    # Reusing the optimize_biscuit_placement logic with additional tracking
    max_values = np.zeros(dough_length + 1)
    placement_tracker = [None] * (dough_length + 1)  # Track biscuit placements

    for position in range(dough_length):
        max_values[position + 1] = max(max_values[position + 1], max_values[position])

        for biscuit_name, biscuit in biscuits.items():
            if can_place_biscuit(biscuit, position, defects):
                end_position = position + biscuit['length']
                if end_position <= dough_length:
                    new_value = max_values[position] + biscuit['value']
                    if new_value > max_values[end_position]:
                        max_values[end_position] = new_value
                        placement_tracker[end_position] = (biscuit_name, position, biscuit['value'])

    # Reconstruct the placement of biscuits
    biscuit_placement = []
    current_position = dough_length
    while current_position > 0:
        placement = placement_tracker[current_position]
        if placement:
            biscuit_name, position, value = placement
            biscuit_placement.append((biscuit_name, position, value))
            current_position = position
        else:
            current_position -= 1

    return list(reversed(biscuit_placement)), max_values[-1]


# Optimizing the biscuit placement
biscuit_placement_details, total_value = optimize_biscuit_placement(biscuits, dough_length, defects_data)
print(f"Total value of biscuits placed: {total_value}")

# Getting detailed placement of biscuits
biscuit_placement_details

Total value of biscuits placed: 760.0


[('Biscuit_1', 0, 12),
 ('Biscuit_1', 8, 12),
 ('Biscuit_0', 16, 6),
 ('Biscuit_1', 20, 12),
 ('Biscuit_1', 28, 12),
 ('Biscuit_0', 36, 6),
 ('Biscuit_0', 41, 6),
 ('Biscuit_3', 45, 8),
 ('Biscuit_0', 50, 6),
 ('Biscuit_0', 54, 6),
 ('Biscuit_3', 58, 8),
 ('Biscuit_3', 63, 8),
 ('Biscuit_3', 68, 8),
 ('Biscuit_1', 73, 12),
 ('Biscuit_1', 81, 12),
 ('Biscuit_1', 89, 12),
 ('Biscuit_0', 99, 6),
 ('Biscuit_3', 103, 8),
 ('Biscuit_3', 108, 8),
 ('Biscuit_0', 114, 6),
 ('Biscuit_0', 118, 6),
 ('Biscuit_0', 122, 6),
 ('Biscuit_3', 126, 8),
 ('Biscuit_3', 131, 8),
 ('Biscuit_0', 136, 6),
 ('Biscuit_0', 140, 6),
 ('Biscuit_3', 144, 8),
 ('Biscuit_1', 149, 12),
 ('Biscuit_1', 158, 12),
 ('Biscuit_3', 167, 8),
 ('Biscuit_0', 172, 6),
 ('Biscuit_1', 176, 12),
 ('Biscuit_0', 186, 6),
 ('Biscuit_3', 190, 8),
 ('Biscuit_3', 195, 8),
 ('Biscuit_3', 200, 8),
 ('Biscuit_3', 205, 8),
 ('Biscuit_3', 210, 8),
 ('Biscuit_0', 215, 6),
 ('Biscuit_3', 219, 8),
 ('Biscuit_3', 224, 8),
 ('Biscuit_1', 229, 12),


In [28]:
# Generate an initial solution
def generate_initial_solution():
    solution = []
    position = 0
    while position < dough_length:
        biscuit_idx = random.randint(0, len(biscuits) - 1)
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            solution.append(biscuit_idx)
            position += biscuit['length']
        else:
            position += 1
    return solution

# Evaluate a solution
def evaluate(solution):
    total_value = 0
    position = 0
    for biscuit_idx in solution:
        biscuit_name = f'Biscuit_{biscuit_idx}'
        biscuit = biscuits[biscuit_name]
        if can_place_biscuit(biscuit, position, defects_data):
            total_value += biscuit['value']
            position += biscuit['length']
        else:
            position += 1
    return total_value

# Generate a neighboring solution
def generate_neighbor(solution):
    neighbor = solution[:]
    change_idx = random.randint(0, len(neighbor) - 1)
    neighbor[change_idx] = random.randint(0, len(biscuits) - 1)
    return neighbor

# Hill climbing algorithm
def hill_climbing(initial_solution):
    current_solution = initial_solution
    current_value = evaluate(current_solution)
    no_improvement_count = 0

    while no_improvement_count < 100:  # Allow some iterations to find improvement
        neighbor = generate_neighbor(current_solution)
        neighbor_value = evaluate(neighbor)

        if neighbor_value > current_value:
            current_solution = neighbor
            current_value = neighbor_value
            no_improvement_count = 0
        else:
            no_improvement_count += 1

    return current_solution, current_value

# Test the local search algorithm
initial_solution = generate_initial_solution()
best_solution, best_value = hill_climbing(initial_solution)
best_solution, best_value

([1,
  1,
  0,
  0,
  3,
  2,
  2,
  0,
  2,
  0,
  1,
  2,
  0,
  3,
  1,
  1,
  3,
  2,
  3,
  0,
  2,
  0,
  0,
  2,
  0,
  2,
  2,
  1,
  2,
  3,
  3,
  0,
  1,
  2,
  0,
  0,
  1,
  2,
  1,
  0,
  1,
  3,
  3,
  1,
  1,
  2,
  3,
  0,
  1,
  1,
  2,
  0,
  2,
  1,
  1,
  1,
  3,
  1,
  0,
  1,
  0,
  2,
  1,
  3,
  0,
  0,
  2,
  2,
  0,
  2,
  1,
  3,
  0,
  1,
  1,
  1,
  2,
  2,
  1,
  1,
  0,
  1,
  3,
  3,
  3,
  3,
  2,
  1,
  0,
  0,
  0,
  3,
  1],
 517)

713